In [7]:
import random
import queue
import concurrent.futures
import math
import numpy as np
from functools import reduce
from threading import Barrier

def share(secret, client_num):
    """
    Input secret     (int): The secret to share.
    Input client_num (int): The number of parties to share the secret to.
    Output     (list[int]): The produced additive shares.

    Produces additive, arithmetic shares based on the number of clients.
    The secret can be reconstructed by simply performing addition on all the shares.
    """
    shares = [0] * client_num
    for i in range(client_num - 1):
        shares[i] = random.randint(1, 6)
    shares[client_num - 1] = secret - sum(shares[:client_num - 1])
    return shares

def generate_shares(binary_list, n):
    """
    Input binary_list (list[int]): The secret to share, represented as a binary number.
    Input n           (int)      : The number of parties to share the secret to.
    Output      (list[list[int]]): The produced binary shares.

    Produces binary shares based on the number of clients.
    The secret can be reconstructed by XORing all the shares.
    """
    shares = []
    current_share = binary_list
    for _ in range(n-1):
        new_share = [random.randint(0, 1) for _ in range(8)]
        current_share = [a ^ b for a, b in zip(current_share, new_share)]
        shares.append(new_share)
    shares.append(current_share)
    return shares

def reconstruct(shares):
    """
    Input shares (list[list[int]]): The binary shares of all parties.
    Output             (list[int]): The recovered secret, represented as a binary number.

    Reconstructs a secret from binary shares, by XORing all the shares.
    """
    result = [0] * 8
    for share in shares:
        result = [a ^ b for a, b in zip(result, share)]
    return result

def toBinary(A):
    neg = False
    if A < 0:
        neg = True
        A = -A
    B = []
    while A > 0:
        B.append(A % 2)
        A = math.floor(A / 2)
    B.append(0)
    if neg:
        c = 0
        for i in range(len(B)):
            if c:
                B[i] = 1 - B[i]
            else:
                c = B[i] == 1
    B.reverse()
    return B

def fromBinary(B):
    if B[0] == 0:
        return reduce(lambda a, b: 2*a + b, B)
    else:
        return -fromBinary([1 - b for b in B]) - 1

def CP(i, x, y, U, V, W, r, r_msb, input_queue, output_queue, barrier):
    """
    The main function each party executes separately.
    """
    
    def A2B(s):
        """
        Input s (int): Input number
        Output  (int): s mod 2

        Performs the A2B protocol, which outputs s mod 2.
        """
        return s % 2
    
    def B2A(s):
        """
        Input s (list[int]): Input shared over binary field.
        Output  (list[int]): Output shared over field F.

        Performs the B2A protocol to convert a binary share to
        an arithmetic share over the field F.
        """
        r_2 = A2B(r)
        c = [bit ^ r_2 for bit in s]

        for q in output_queue:
            q.put(c)
        
        for _ in range(len(output_queue)):
            c = [bit1 ^ bit2 for (bit1, bit2) in zip(c, input_queue.get())]
        
        if i == 0:
            return [bit + r - 2 * bit * r for bit in c]
        else:
            return [r - 2 * bit * r for bit in c]

    def mul_number(s, d):
        """
        Multiplication function based on Beaver triple.
        """
        D = s - U
        E = d - V

        # Sending and receiving intermediate results
        for q in output_queue:
            q.put((D, E))

        DE_values = [(D, E)] + [input_queue.get() for _ in range(len(output_queue))]

        D_sum = sum(D for D, E in DE_values)
        E_sum = sum(E for D, E in DE_values)

        Z = W + (D_sum * V) + (U * E_sum)

        if i == 0:
            Z += (D_sum * E_sum)
        return Z
    
    def o1(d2, d1):
        """
        Carry propagation operator o.
        Computes a new (p, g) pair from two input pairs:
        (p, g) = (p2, g2) o (p1, g1) = (p1 & p2, g2 | (p2 & g1))
        """
        (p2, g2) = d2
        (p1, g1) = d1
        return (p1 & p2, g2 | (p2 & g1))

    def o(d2, d1):
        """
        Input d2 ((int, int)): Input pair (p2, g2) shared over field F
        Input d1 ((int, int)): Input pair (p1, g1) shared over field F
        Output   ((int, int)): Output pair (p, g) shared over field F

        Carry propagation operator o.
        Computes a new (p, g) pair from two input pairs:
        (p, g) = (p2, g2) o (p1, g1) = (p1 * p2, g2 + p2 * g1)
        """
        (p2, g2) = d2
        (p1, g1) = d1
        return (mul_number(p1, p2), g2 + mul_number(p2, g1))
    
    def PreOpL1(i, A):
        i = 1
        j = 2
        while j <= len(A):
            for k in range(math.floor(len(A) / j)):
                p = i + (k + 1) * j
                for q in range(i):
                    A[p+q] = o(i, A[p-1], A[p+q])
            i = j
            j *= 2
        return A
    
    def PreOpL(A):
        """
        Input A (list[(int, int)]): Input prepared for carry propagation (see prepareCarry function)
        Output  (list[(int, int)]): Output list of (P, G) pairs

        Performs the PreOpL2 protocol with the carry propagation operator (see function o).
        """
        k = len(A)
        kh = math.floor(k / 2)

        P = A
        P[0] = A[0]

        if k > 1:
            U = [0] * kh
            for j in range(1, 1+kh):
                U[j-1] = o(A[2*j - 1], A[2*j - 2])
            V = PreOpL(U)
            for j in range(1, 1+kh):
                P[2*j - 1] = V[j - 1]
            for j in range(2, 1+kh):
                P[2*j - 2] = o(A[2*j - 2], V[j - 2])
        
        return P

    def PreOpC(D):
        """
        Computes the carry bits from the input (p, g) pairs.
        The input consists of a (p, g) pair for each pair of bits to add.
        Each resulting pair is computed as (Pi, Gi) = (pi, gi) o (P[i-1], G[i-1])
        with (P0, G0) = (p0, g0).
        From each resulting pair, the carry bit can be obtained with ci = Gi.
        """
        p = D[0]
        P = [p]
        for i in range(1,len(D)):
            p = o(D[i], p)
            P.append(p)
        return P
    
    def prepareCarry(A, B):
        """
        Input A (list[int]): Input shared over field F
        Input B (list[int]): Input shared over field F
        Output  (list[(int, int)]): Prepared output as a list of (p, g) pairs

        Prepares the inputs A and B for the carry propagation operator.
        The output pairs generated are for each pair of input bits:
        (p, g) = (a + b - 2ab, ab)
        """
        return [(a + b - 2 * ab, ab) for (a, b) in zip(A, B) if (ab := mul_number(a, b))]

    def addbitwise(A, B):
        """
        Input A (list[int]): Input shared over field F
        Input B (list[int]): Input shared over field F
        Output  (list[int]): Addition of A+B shared over field F

        Performs bitwise addition on the given bit strings.
        The input parameters A and B are arrays of integers representing bits.
        The leftmost bits (A[0] and B[0]) represent the most significant bits.
        The output is a single array of bits representing the value A+B.
        """
        k = len(A)

        # Reverse A and B so that index 0 is LSB
        A.reverse()
        B.reverse()

        # Calculate carry bits
        D = prepareCarry(A, B)
        C = PreOpL(D)

        # Perform addition with carry bits
        S = [A[0] + B[0] - 2*C[0][1]]
        for i in range(1, k):
            S.append(A[i] + B[i] + C[i-1][1] - 2*C[i][1])
        S.reverse()

        return S
    
    def CarryOutAux(D):
        """
        Input D (list[(int, int)]): Input prepared for carry propagation (see prepareCarry function)
        Output  ((int, int))      : Computed (p, g) pair with the outgoing carry.

        Performs the protocol CarryOutAux used by the CarryOutCin protocol.
        """
        k = len(D)
        if k > 1:
            U = []
            for j in range(1, 1 + math.floor(k/2)):
                U.append(o(D[2*j - 1], D[2*j - 2]))
            return CarryOutAux(U)
        else:
            return D[0]
    
    def CarryOutLin(D):
        return PreOpC(D)[len(D)-1]
    
    def CarryOutCin(A, B, c):
        """
        Input A (list[int]): Input shared over field F
        Input B (list[int]): Input shared over field F
        Input c (int)      : Incoming carry bit
        Output  (int)      : Outgoing carry shared over field F

        Calculates the carry-out bit from the inputs A and B
        and an incoming carry bit c.
        """
        # Reverse A and B so that index 0 is LSB
        A.reverse()
        B.reverse()

        # Calculate carry bits
        D = prepareCarry(A, B)

        # Process incoming carry bit
        (p, g) = D[0]
        D[0] = (p, g + c*p)

        (_, g) = CarryOutAux(D)
        return g
    
    def BitLT0(i, A, B):
        B = [1 - b for b in B]
        return 1 - CarryOutCin(A, B, 1)
    
    def BitLT(A, B):
        """
        Input A (list[int]): Input shared over binary field
        Input B (list[int]): Input shared over binary field
        Output        (int): Result A < B, shared over field F

        Performs the BitLT protocol.
        Bitwise compares the two input strings A and B, to
        determine A < B.
        """
        AF = B2A(A)                  # Share A  over field F
        BF = B2A([1 - b for b in B]) # Share B' over field F where b' = 1 - b (invert all bits in B)
        c = CarryOutCin(AF, BF, 1)

        if i == 0:
            return 1 - c
        else:
            return -c
        
    def MSB(A):
        k = len(A)
        kpow = pow(2, k-1)
        R = B2A(r_msb)
        c = addbitwise(A, R)
        print(A, R, c)

        for q in output_queue:
            q.put(c)
        
        for _ in range(len(output_queue)):
            c = [a + b for (a, b) in zip(c, input_queue.get())]
        
        c = c[1:]     # Step 3: c mod 2^(k-1)
        r = r_msb[1:] # Step 3: r' = r[0:k-2]
        r_2 = [A2B(ri) for ri in r] # Step 4
        u = [BitLT(c, r)]
        print("u", u)
        uF = B2A(u)
        
        print("c", c)

    # Share x and y over field F
    xF = B2A(x)
    yF = B2A(y)
    print(" ")

    MSB(xF)

    add = addbitwise(xF, yF)
    print("add", add)

    carry = CarryOutCin(xF, yF, 1)
    lt = BitLT(x, y)
    print("carry", carry, lt)
    return (add, lt)

def SPDZ_prepare(beta1, beta2, group_size, bit_num):
    """
    Function to produce Beaver triple shares.
    """
    U, V = random.randint(3, 6), random.randint(3, 6)
    W = U * V
    r = random.randint(0, 1)
    r_msb = [random.randint(0, 1) for _ in range(bit_num)]

    x_shares = generate_shares(beta1, n)
    y_shares = generate_shares(beta2, n)
    r_msb_shares = generate_shares(r_msb, n)
    return share(U, group_size), share(V, group_size), share(W, group_size), x_shares, y_shares, share(r, group_size), r_msb_shares
def SPDZ_execute(preparation_data, group_size):
    """
    Function to execute the SPDZ protocol.
    """
    with concurrent.futures.ThreadPoolExecutor() as executor:
        barrier = Barrier(group_size)
        U_shares, V_shares, W_shares, x_shares, y_shares, r_shares, r_msb_shares = preparation_data
        queues = [queue.Queue() for _ in range(group_size)]
        futures = [
            executor.submit(
                CP, j, x_shares[j], y_shares[j], U_shares[j], V_shares[j], W_shares[j], r_shares[j], r_msb_shares[j],
                queues[j], [queues[k] for k in range(group_size) if k != j], barrier
            ) for j in range(group_size)
        ]

        group_results = [f.result() for f in futures]
        add_results = [add for (add, _) in group_results]
        lt_results = [lt for (_, lt) in group_results]

        print("ADD: Final results share of each party:", add_results)
        sum_total = np.sum(add_results, axis=0).tolist()
        print("Final Result:", sum_total)

        print("LT: Final results share of each party:", lt_results)
        sum_total = np.sum(lt_results, axis=0).tolist()
        print("Final Result:", sum_total)

# Main execution
if __name__ == "__main__":
    n = 5  # Number of parties
    bit_num = 8

    input2 = [0, 0, 0, 1, 0, 1, 0, 0] # 20
    input1 = [0, 0, 0, 0, 1, 0, 1, 0] # 10

    preparation_data = SPDZ_prepare(input1, input2, n, bit_num)
    SPDZ_execute(preparation_data, n)


  
 
 
 

[1, -1, 1, -1, 1, 1, 1, 1] [-1, -1, -1, -1, 1, -1, -1, -1] [189, 34, 120, 38, 125, 51, 95, 58]
[3, -3, 3, -3, 3, 3, 3, 3] [-3, -3, -3, -3, 3, -3, -3, -3] [69, 12, 44, 20, 41, 19, 29, 22]
[-10, 10, -10, 10, -10, -10, -10, -10] [10, 10, 10, 10, -10, 10, 10, 10] [-402, -70, -260, -94, -250, -118, -184, -128]
[4, -3, 4, -3, 4, 4, 4, 4] [-3, -3, -3, -3, 4, -3, -3, -3] [28, 4, 23, 10, 10, 17, 4, 13]
[2, -2, 2, -2, 2, 2, 2, 2] [-2, -2, -2, -2, 2, -2, -2, -2] [117, 21, 74, 27, 75, 31, 56, 36]
u 37
u 103
u 64
u 9
u -212


TypeError: 'int' object is not iterable